# Creating training images
This notebook creates training images from a video, by cutting into the back of the rower only, using the shoulders, hips and nose.
Change the **images_per_frame** variable to reduce the amount of images created by the given video.
Keep in mind you need to multiply the variables by the size of the video.

## Images to train a bad back posture
Images were extracted from youtube videos with screenshots on the back posture. Plus some mp4 videos, that mainly provide good posture, as mentioned, a lot of bad posture were extracted directly from youtube.

## Train-test
Currently we have a train and test folder inside img. Inside both folders we have **good** and **bad** folders, that include images for good and bad postures. Currently we have inbalanced image dataset.

### What the notebooks does?
The notebook creates images from videos, then you can decide wether that is a good posture or not.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
mp_drawing = mp.solutions.drawing_utils
# We want the pose estimation
mp_pose = mp.solutions.pose

In [ ]:
# Video
cap = cv2.VideoCapture('../videos/rowing4.mp4')

count = 0
images_per_frame = 5
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:  
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            right_shoulder = landmarks[12]
            right_hip = landmarks[24]
            left_shoulder = landmarks[11]
            left_hip = landmarks[23]
            nose = landmarks[0]
            max_x = max(right_shoulder.x, left_shoulder.x, right_hip.x, left_hip.x)
            max_x = max_x + (max_x * 0.2)
            min_x = min(right_shoulder.x, left_shoulder.x, right_hip.x, left_hip.x)
            min_x = min_x - (min_x * 0.2)
            min_y = min(right_hip.y, left_hip.y)
            min_y = min_y + (min_y * 0.1)
            max_y = nose.y - (nose.y * 0.2)
        except:
            pass
        
        if count % images_per_frame == 0:
            cv2.imwrite("img/{}.png".format(count), image[int(max_y*720):int(min_y*720), int(min_x*1280):int(max_x*1280)])
        count += 1


In [ ]:
# Rename the images
img_list = os.listdir("../img/good")
count = 0
for img in img_list:
    os.rename("../img/good/{}".format(img), "../img/good/good_{}.png".format(count))
    count += 1